In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np
import sqlite3

In [2]:
#Checking for Vanderbuilt
query = """
SELECT *
FROM npi_nppes
WHERE "Provider Organization Name (Legal Business Name)" LIKE '%Vanderbilt%'
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    finding_vanderbilt = pd.read_sql(query, db)
    
finding_vanderbilt

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1124017959,2.0,VANDERBILT UNIVERSITY MEDICAL SCHOOL,None,None,None,None,None,None,VANDERBILT UNIVERSITY DIVISION OF MEDICAL GENE...,DD 2205 MCN,NASHVILLE,TN,372320001.0,170300000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,170300000X,"Genetic Counselor, MS",37232,34980
1,1598738205,2.0,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,None,None,None,None,None,None,2201 CHILDRENS WAY,None,NASHVILLE,TN,372123164.0,283X00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,283X00000X,Rehabilitation Hospital,37212,34980
2,1992770119,2.0,VANDERBILT MEDICAL CENTER,None,None,None,None,None,None,S-3414 MEDICAL CENTER NORTH,C/O INTERNAL MEDICINE DEPT,NASHVILLE,TN,372320001.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37232,34980
3,1952356065,2.0,VANDERBILT UNIVERSITY,None,None,None,None,None,None,3601 THE VANDERBILT CLINIC,None,NASHVILLE,TN,372325100.0,204E00000X,N,204F00000X,N,207L00000X,N,207P00000X,N,207Q00000X,N,207V00000X,N,207W00000X,N,207X00000X,N,207Y00000X,N,208000000X,N,2084N0400X,N,2084P0800X,N,208600000X,N,208800000X,N,207R00000X,Y,207R00000X,Internal Medicine,37232,34980
4,1821030842,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,500 PARK AVENUE,None,LEBANON,TN,370873721.0,273R00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,273R00000X,Psychiatric Unit,37087,34980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,1265210843,2.0,"VANDERBILT INTEGRATED PROVIDERS, LLC",None,None,None,None,None,None,1440 CEDAR LN STE 200,None,TULLAHOMA,TN,373882486.0,261QR1300X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Non

In [13]:
#table with the correct Vanderbilt organization
query = """
SELECT *
FROM npi_nppes
WHERE "Provider Organization Name (Legal Business Name)" = 'VANDERBILT UNIVERSITY MEDICAL CENTER'
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    The_vanderbilt = pd.read_sql(query, db)
    
The_vanderbilt

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1821030842,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,500 PARK AVENUE,None,LEBANON,TN,370873721.0,273R00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,273R00000X,Psychiatric Unit,37087,34980
1,1215979190,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,500 PARK AVENUE,None,LEBANON,TN,370873721.0,273Y00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,273Y00000X,Rehabilitation Unit,37087,34980
2,1306889597,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,1411 W. BADDOUR PARKWAY,None,LEBANON,TN,370872513.0,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37087,34980
3,1700800877,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,2200 CHILDRENS WAY,ROOM 2106A,NASHVILLE,TN,372329650.0,3336C0003X,N,3336S0011X,N,3336I0012X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3336I0012X,Pharmacy,37232,34980
4,1700950284,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,1215 21ST AVE S,3108 MEDICAL CENTER EAST,NASHVILLE,TN,372320014.0,282NC0060X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282NC0060X,General Acute Care Hospital,37232,34980
5,1558408633,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,1601 23RD AVE S,None,NASHVILLE,TN,372123133.0,283Q00000X,N,282N00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,282N00000X,General Acute Care Hospital,37212,34980
6,1396882205,2.0,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,1211 MEDICAL CENTER DRIVE,None,NASHVILLE,TN,372320004.0,273R00000X,N,282N00000X,N,282NC2000X,N,283Q00000X,N,291U00000X,N,3416A08

**Research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).**

In [39]:
#Finding TriStar Centennial Medical Center ONLY in Nashville area
query = """
SELECT "Provider Organization Name (Legal Business Name)","NPI","Provider First Line Business Practice Location Address", SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "CBSA" = '34980' AND "Provider Organization Name (Legal Business Name)" LIKE '%TriStar%'
GROUP BY 1,2,3
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    finding_tristar = pd.read_sql(query, db)
    
finding_tristar

,Provider Organization Name (Legal Business Name),NPI,Provider First Line Business Practice Location Address,Amount of Patients,Total_referrals
0,"TRISTAR BONE MARROW TRANSPLANT, LLC",1861869596,2410 PATTERSON ST STE 500,1453,4389
1,TRISTAR CARDIOVASCULAR SURGERY LLC,1770851552,2400 PATTERSON ST,9030,13613
2,"TRISTAR FAMILY CARE, LLC",1609326040,313 N MAIN ST FL 2,8498,17794
3,"TRISTAR GYNECOLOGY ONCOLOGY, LLC",1003152208,330 23RD AVE N,4557,7119
4,"TRISTAR JOINT REPLACEMENT INSTITUTE, LLC",1164834057,2400 PATTERSON ST,20673,31116
5,TRISTAR MEDICAL GROUP - CENTENNIAL PRIMARY CAR...,1679835623,330 23RD AVE N,7169,17381
6,"TRISTAR MEDICAL GROUP - LEGACY HEALTH, LLC",1821548280,"2400 PATTERSON STREET, SUITE 418",2414,5423
7,"TRISTAR RADIATION ONCOLOGY, LLC",1952652414,2410 PATTERSON ST,711,2041


In [38]:
#Finding TriStar Centennial Medical Center total organizations
query = """
SELECT "Provider Organization Name (Legal Business Name)","NPI", SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Provider Organization Name (Legal Business Name)" LIKE '%TriStar%'
GROUP BY 1
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    finding_tristar_all = pd.read_sql(query, db)
    
finding_tristar_all

,Provider Organization Name (Legal Business Name),NPI,Amount of Patients,Total_referrals
0,"TRISTAR BONE MARROW TRANSPLANT, LLC",1861869596,1453,4389
1,TRISTAR CARDIOVASCULAR SURGERY LLC,1770851552,9030,13613
2,"TRISTAR CARE CENTER, INC.",1083793822,580,2700
3,"TRISTAR FAMILY CARE, LLC",1609326040,16996,35588
4,"TRISTAR GYNECOLOGY ONCOLOGY, LLC",1003152208,4557,7119
5,"TRISTAR JOINT REPLACEMENT INSTITUTE, LLC",1164834057,20673,31116
6,TRISTAR MEDICAL GROUP - CENTENNIAL PRIMARY CAR...,1679835623,7169,17381
7,"TRISTAR MEDICAL GROUP - LEGACY HEALTH, LLC",1821548280,2414,5423
8,TRISTAR MEDICAL INDUSTRIAL GROUP,1164834495,15,64
9,"TRISTAR MEDICAL LAB, INC",1730195215,9124,28551


In [55]:
tristar_npi = tuple(finding_tristar_all['NPI'].tolist())

In [61]:
tristar_npi

(1861869596,
 1770851552,
 1083793822,
 1609326040,
 1003152208,
 1164834057,
 1679835623,
 1821548280,
 1164834495,
 1730195215,
 1255579728,
 1952652414)

In [57]:
finding_tristar_all['NPI']

0     1861869596
1     1770851552
2     1083793822
3     1609326040
4     1003152208
5     1164834057
6     1679835623
7     1821548280
8     1164834495
9     1730195215
10    1255579728
11    1952652414
Name: NPI, dtype: int64

In [60]:
db.close()

In [62]:
query = """
SELECT COUNT(*)
FROM npi_nppes 

"""
with sqlite3.connect('../data/npi.sqlite') as db:
    check = pd.read_sql(query, db)
check

,COUNT(*)
0,8989950


In [63]:
query = """
SELECT "NPI", "Provider Organization Name (Legal Business Name)","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "CBSA" = '34980' AND to_npi in (1861869596, 1770851552, 1083793822, 1609326040,  1821548280,  1164834495, 1730195215, 1255579728, 1952652414)
    

GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    to_tristar = pd.read_sql(query, db)
    
to_tristar

KeyboardInterrupt: 

In [ ]:
query = f"""
SELECT "NPI", "Provider Organization Name (Legal Business Name)","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "CBSA" = '34980' AND to_npi in {tristar_npi}
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    to_tristar = pd.read_sql(query, db)
    
to_tristar

***Consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?***

In [46]:
# family medicine doctors that vanderbilt should reach out to get their patient referrals.
query = """
SELECT "NPI", "Provider Organization Name (Legal Business Name)","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Classification" = 'Family Medicine' AND "CBSA" = '34980' AND to_npi in (1861869596, 1770851552, 1083793822, 1609326040, 1003152208, 1164834057,  1679835623, 1821548280, 1164834495, 1730195215, 1255579728, 1952652414)
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    fam_med_to_vanderbilt = pd.read_sql(query, db)
    
fam_med_to_vanderbilt

,NPI,Provider Organization Name (Legal Business Name),Fullname,Amount of Patients,Total_referrals
0,1184620460,None,JAMES ANDERSON MD,805,2882
1,1568464873,DICKSON MEDICAL ASSOCIATES PC,None,156,216
2,1578766697,None,CHRIS MEHR MD,59,68


In [47]:
# Orthopaedic surgeons that vanderbilt should reach out to get their patient referrals.
query = """
SELECT "NPI", "Provider Organization Name (Legal Business Name)","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Classification" = 'Orthopaedic Surgery' AND "CBSA" = '34980' AND to_npi in (1861869596, 1770851552, 1083793822, 1609326040, 1003152208, 1164834057,  1679835623, 1821548280, 1164834495, 1730195215, 1255579728, 1952652414)
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    ortho_surg_to_vanderbilt = pd.read_sql(query, db)
    
ortho_surg_to_vanderbilt

,NPI,Provider Organization Name (Legal Business Name),Fullname,Amount of Patients,Total_referrals
0,1942265400,None,DAVID DEBOER MD,719,1596
1,1841255346,None,MICHAEL CHRISTIE MD,709,1499
2,1609831916,None,JEFFERSON MORRISON MD,635,1357
3,1265556674,None,JEFFREY HODRICK M.D.,597,1297
4,1003013947,None,ROBERT OTTO M.D.,193,472
5,1538153937,TENNESSEE ORTHOPAEDIC ALLIANCE PA,None,148,228
6,1972577351,PREMIER ORTHOPAEDICS & SPORTS MEDICINE PLC,None,47,83


In [30]:
#finding the family medicine providers in the Nashville area that refers the most patients.
query = """
SELECT "NPI","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, "Classification", SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Classification" = 'Family Medicine' AND "CBSA" = '34980' AND "Entity Type Code" = 01
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    family_med = pd.read_sql(query, db)
    
family_med

,NPI,fullname,Classification,Amount of Patients,Total_referrals
0,1184612921,DONALD VOLLMER M.D.,Family Medicine,20756,47832
1,1215029897,RICHARD KENNEDY M.D.,Family Medicine,16173,35366
2,1861451338,JOSEPH RUPARD M D,Family Medicine,8026,22458
3,1619945292,KEITH HILL MD,Family Medicine,18371,19754
4,1376635052,JAMES COUCH M.D.,Family Medicine,8077,18784
...,...,...,...,...,...
397,1902858574,MICHAEL BERNUI D.O.,Family Medicine,45,66
398,1790710978,HAROLD NEVELS MD,Family Medicine,48,58
399,1639221641,CYNTHIA COLLINS MD,Family Medicine,18,57
400,1487941522,PAUL NGO D.O,Family Medicine,23,56


In [31]:
##finding the orthopaedic surgeon providers in the Nashville area that refers the most patients.
query = """
SELECT "NPI","Provider First Name" ||' '|| "Provider Last Name (Legal Name)" ||' '|| "Provider Credential Text" AS Fullname, "Classification", SUM("patient_count") AS "Amount of Patients", SUM("transaction_count") AS Total_referrals
FROM npi_nppes AS nppes
INNER JOIN hop_npi AS npi
ON nppes.NPI = npi.from_npi
WHERE "Classification" = 'Orthopaedic Surgery' AND "CBSA" = 34980 AND "Entity Type Code" = 01
GROUP BY 1,2,3
ORDER BY 5 DESC;
"""
with sqlite3.connect('../data/npi.sqlite') as db:
    ortho_surgeon = pd.read_sql(query, db)
    
ortho_surgeon

,NPI,fullname,Classification,Amount of Patients,Total_referrals
0,1649271206,ROY TERRY M.D.,Orthopaedic Surgery,8860,14368
1,1982656609,JOSEPH WADE MD,Orthopaedic Surgery,5665,9062
2,1548259864,WALTER DUBE M.D.,Orthopaedic Surgery,4425,8491
3,1184677346,JEFFREY ADAMS M.D.,Orthopaedic Surgery,4514,7880
4,1720077910,PAUL THOMAS MD,Orthopaedic Surgery,4581,7380
...,...,...,...,...,...
181,1396907705,ETHAN KELLUM MD,Orthopaedic Surgery,65,100
182,1639519903,CASEY DAVIDSON M.D.,Orthopaedic Surgery,75,93
183,1588701999,JOHN BYRD MD,Orthopaedic Surgery,53,92
184,1720354343,ADAM COCHRAN MD,Orthopaedic Surgery,47,77


In [51]:
query = """
SELECT * 
FROM npi_nppes as np
join hop_npi as n
on np.NPI = n.from_npi
WHERE np.CBSA = 34980

"""
with sqlite3.connect('../data/npi.sqlite') as db:
    nppes_nashville = pd.read_sql(query, db)

In [52]:
nppes_nashville.to_csv("../data/nppes_nashville.csv")